# 오브젝트 검출

In [1]:
import numpy as np
import cv2
import glob
import os
import sys
from matplotlib import pyplot as plt

In [2]:
def nothing(x):
    pass

밝기대비를 늘려주는 이미지 전처리 함수를 정의한다.


In [3]:
def contrast_stretching(src):
    hist, bins = np.histogram(img.flatten(), 256, [0,256])

    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max() / cdf.max()

    img_cs = np.zeros((img.shape[0], img.shape[1]), dtype='uint8')

    pxmin = np.min(img)
    pxmax = np.max(img)

    for i in range(img.shape[0]):
        for j  in range(img.shape[1]):
            img_cs[i,j] = 255 * (img[i,j]-pxmin) / (pxmax-pxmin)

    # result histogram
    hist_cs, bin_cs = np.histogram(img_cs.flatten(), 256, [0,256])

    cdf_cs = hist_cs.cumsum()
    cdf_cs_normalized = cdf_cs * hist_cs.max() / cdf_cs.max()

    return img_cs



라벨링(contour) 이미지와 canny edge이미지를 AND연산한다.

In [9]:
path = "/home/minkyung/source/images/shot/04/"
img_seq = os.listdir(path)

test = 'Contour test'

print(img_seq)
for iname in img_seq:
    n_nonzero = 0
    n_nonzero_pre = 0
    img = cv2.imread(path+iname, cv2.IMREAD_GRAYSCALE)
    
    img_cs = contrast_stretching(img)

    cv2.namedWindow(test)
    cv2.createTrackbar('canny_minval', test, 0, 255, nothing)
    cv2.createTrackbar('canny_maxval', test, 0, 255, nothing)

    cv2.createTrackbar('thresh_val', test, 0, 255, nothing)
    #cv2.createTrackbar('erode_ksize', test, 1, 9, nothing)
    #cv2.createTrackbar('dilate_ksize', test, 1, 9, nothing)
    cv2.createTrackbar('ksize', test, 0, 9, nothing)
    cv2.createTrackbar('dilate_iter', test, 1, 5, nothing)

    while True:
        minval = cv2.getTrackbarPos('canny_minval', test)
        maxval = cv2.getTrackbarPos('canny_maxval', test)

        thresh_val = cv2.getTrackbarPos('thresh_val', test)
        #erode_ksize = cv2.getTrackbarPos('erode_ksize', test)
        #dilate_ksize = cv2.getTrackbarPos('dilate_ksize', test)
        ksize = cv2.getTrackbarPos('ksize', test)
        dilate_iter = cv2.getTrackbarPos('dilate_iter', test)

        #
        # Contour
        (T, img_bin) = cv2.threshold(img_cs, thresh_val, 255, cv2.THRESH_BINARY)
        
        kernel = np.ones((ksize, ksize), 'uint8')
        img_erode = cv2.erode(img_bin, kernel, iterations=1)
        img_dilate = cv2.dilate(img_erode, kernel, iterations=dilate_iter)

        #
        # Edges
        edges = cv2.Canny(img_cs, minval, maxval)

        #
        # Merge
        result = np.bitwise_and(edges, img_dilate)
        
        n_nonzero = np.count_nonzero(result)
        #cv2.imshow('Edge test', edges)

        scale_percent = 50
        width = int(result.shape[1] * scale_percent /  100)
        height = int(result.shape[0] * scale_percent / 100)
        dsize = (width, height)
        img_resize = cv2.resize(result, dsize)

        cv2.imshow(test, img_resize)

        end_key = cv2.waitKey(1)


        if end_key == 27:
            print('canny_minval=', minval)
            print('canny_maxval=', maxval)
            print('thresh_val=', thresh_val)
            print('ksize=', ksize)
            print('dilate_iter=', dilate_iter)
            print()
            break;
        elif end_key == 99:
            print('count nonzero=', n_nonzero)
    cv2.destroyAllWindows()

    #plt.axis('off')
    #plt.imshow(img, cmap='gray'), plt.title(iname)

    #plt.show()

['impacttigger.jpg', 'readyball.jpg']
count nonzero= 657
canny_minval= 98
canny_maxval= 115
thresh_val= 30
ksize= 9
dilate_iter= 4

canny_minval= 0
canny_maxval= 0
thresh_val= 0
ksize= 0
dilate_iter= 1

